# Muslim Faith Trends Youth Data Analysis

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import os
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Fetch the Google Service Account credentials from GitHub Secrets
google_credentials = json.loads(os.environ.get("GOOGLE_SERVICE_ACCOUNT"))

# Set up Google Sheets API
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_dict(google_credentials, scope)
client = gspread.authorize(credentials)

# Open the Google Sheets file
file_id = "1K7w6NrLIHfFE6eed6nb03ZD4Yin_WAa5Cu49g1z_YyM"  
spreadsheet = client.open_by_key(file_id)
worksheet = spreadsheet.sheet1

# Load data into a DataFrame
data = pd.DataFrame(worksheet.get_all_records())

In [4]:
def analyze_column(column_name, data, recording_schema, figure_width=10):
    """
    Analyzes a column from the data, visualizes its distribution, and displays the question text.

    Parameters:
        column_name (str): The name of the column to analyze.
        data (DataFrame): The DataFrame containing the column.
        recording_schema (DataFrame): The schema containing column mappings and descriptions.

    Returns:
        DataFrame: A DataFrame with Response, Count, and Percentage.
        Visualization: A bar chart of the distribution.
    """
    # Fetch the description and mapping from the recording schema
    description = recording_schema.loc[recording_schema['column_name'] == column_name, 'item_label'].values[0]
    mapping_str = recording_schema.loc[recording_schema['column_name'] == column_name, 'mapping'].values[0]
    beliefs_mapping = eval(mapping_str)  # Convert string to dictionary

    # Ensure column is numeric
    data[column_name] = pd.to_numeric(data[column_name], errors='coerce')

    # Count the responses
    response_counts = data[column_name].value_counts().sort_index()  # Sort by index to align with mapping
    response_percentages = (response_counts / response_counts.sum()) * 100

    # Map numeric values to descriptive labels
    mapped_responses = response_counts.index.map(beliefs_mapping)

    # Create a DataFrame with counts and percentages
    response_stats = pd.DataFrame({
        'Response': mapped_responses,
        'Count': response_counts.values,
        'Percentage': response_percentages.values
    })

    # Display the question text
    print(f"Question: {description}\n")

    # Plot the responses as a bar chart
    plt.figure(figsize=(figure_width, 5))
    plt.bar(response_stats['Response'], response_stats['Count'], color='skyblue')
    plt.title(description, fontsize=14)
    plt.ylabel("Count", fontsize=12)
    plt.xlabel("Response", fontsize=12)

    # Add percentage labels on top of bars
    for i, val in enumerate(response_stats['Count']):
        plt.text(i, val + 1, f"{val} ({response_stats['Percentage'].iloc[i]:.1f}%)", ha='center', fontsize=10)

    plt.tight_layout()
    plt.show()

    return response_stats

In [5]:
def analyze_columns(prefix, data, recording_schema, figsize=15, title="Cumulative Response Analysis"):
    """
    Analyze and plot cumulative responses for columns starting with a given prefix.
    
    Parameters:
        prefix (str): Prefix to filter columns in recording_schema.
        data (DataFrame): Data containing responses.
        recording_schema (DataFrame): Schema containing column names and descriptions.
        figsize (int, optional): Figure size for the plot (default is 15).
        title (str, optional): Title for the graph (default is "Cumulative Response Analysis").
    
    Returns:
        DataFrame: A DataFrame with counts and percentages for each item.
    """
    # Filter only the relevant columns based on the prefix
    relevant_schema = recording_schema[recording_schema['column_name'].str.startswith(prefix)]

    # Calculate counts and percentages
    stats = []
    for _, row in relevant_schema.iterrows():
        column = row['column_name']
        description = row['item_label']
        
        # Ensure column is numeric
        data[column] = pd.to_numeric(data[column], errors='coerce')
        
        count = data[column].sum()  # Count where "Selected" (1)
        total = data[column].count()  # Total non-missing values
        percentage = (count / total) * 100
        stats.append({"Source": description, "Count": count, "Percentage": percentage})

    # Convert stats to a DataFrame and sort by percentage
    stats_df = pd.DataFrame(stats).sort_values(by="Percentage", ascending=False)

    # Display the stats as a table
    display(stats_df)  # Use display to visually show the DataFrame

    # Plot the stats using percentages on the x-axis
    plt.figure(figsize=(figsize, 8))
    plt.barh(stats_df["Source"], stats_df["Percentage"], color="skyblue")
    plt.xlabel("Percentage (%)", fontsize=12)
    plt.title(title, fontsize=14)

    # Add percentage labels to the bars
    for i, percentage in enumerate(stats_df["Percentage"]):
        plt.text(percentage + 0.1, i, f"{percentage:.1f}%", va='center', fontsize=10)

    plt.gca().invert_yaxis()  # Reverse the order for better readability
    plt.tight_layout()
    plt.show()

    return stats_df

In [6]:
recording_schema = pd.read_csv("recording_schema.csv")

In [7]:
from ipywidgets import interact, widgets

def select_column(column_name, figure_width):
    return analyze_column(column_name, data, recording_schema, figure_width)

# Create the interact widget
interact(
    select_column,
    column_name=list(recording_schema['column_name']),
    figure_width=widgets.IntSlider(value=15, min=5, max=30, step=1, description='Figure Width')
);

interactive(children=(Dropdown(description='column_name', options=('practices_1', 'practices_2', 'practices_3'…